In [1]:
%cd H:/SINS dataset

H:\SINS dataset


In [2]:
seed = 1356

import numpy as np
np.random.seed(seed)
from tensorflow import set_random_seed
set_random_seed(seed)

import sklearn
import cv2
import random
import math
import os
import datetime

from itertools import chain
from collections import Counter
from sklearn.metrics import f1_score, confusion_matrix

# constants
img_folder = 'audio'
img_name = ['_pressure.png', '_spec1.png', '_spec2.png', '_spec3.png', '.wav']

im_size = 64
im_size_flat = im_size * im_size
n_labels = 9
n_channels = 1 # grayscale
sd = np.sqrt(2) / np.sqrt(im_size_flat)

req_improve = 500
update_cnt = 50

c:\users\jerem_000\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, LSTM
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import optimizers

Using TensorFlow backend.


In [4]:
import librosa

afs, labels, _ = zip(*[line.rstrip('\n').split('\t') for line in open('meta.txt').readlines()])

In [5]:
def get_folds(index, n_folds = 4):
    afs, labels, _ = zip(*[line.rstrip('\n').split('\t') for line in open('meta.txt').readlines()])
    afs_imlinks = [img_folder + af[5:-4] + img_name[index] for af in afs]
        
    # get smallest no. of classes
    mincnt = Counter(labels).most_common()[-1][1]
    
    afs_label_sep = {}
    for (af, label) in zip(afs_imlinks, labels):
        if label in afs_label_sep:
            afs_label_sep[label].append(af)
        else:
            afs_label_sep[label] = [af]

    afs_label_sep_sampled = {label:random.sample(afs_label_sep[label], mincnt) for label in sorted(afs_label_sep.keys())}
    
    # partition the graph into n_folds partitions for cross-validation
    folds = {fold+1:[] for fold in range(n_folds)}
    samples = int(math.ceil(mincnt / n_folds))

    for label in sorted(afs_label_sep_sampled.keys()):
        random.shuffle(afs_label_sep_sampled[label])
        for fold in range(n_folds):
            folds[fold+1] += [(af, label) for af in afs_label_sep_sampled[label][fold*samples:(fold+1)*samples]]

    return list([folds[key] for key in sorted(folds.keys())])
#     return list(folds.values())

# 0-indexed get training/testing sets
def get_sets(folds, fold = -1):
    _, labels, _ = zip(*[line.rstrip('\n').split('\t') for line in open('meta.txt').readlines()])
    sces = sorted(list(set(labels)))
    sce_int_map = {sce:i+1 for i, sce in enumerate(sces)}
    int_sce_map = {sce_int_map[i]:i for i in sce_int_map.keys()}
        
    ex_fold = folds[fold]
    ot_fold = folds[0:fold] + folds[fold+1:]
    chainfold = list(chain.from_iterable(ot_fold))
    random.shuffle(ex_fold)
    random.shuffle(chainfold)
    trX, trY = zip(*[(af, label) for (af, label) in chainfold])
    tX, tY = zip(*[(af, label) for (af, label) in ex_fold])
        
    trY = np.array([[1 if int_sce_map[c+1] == label else 0 for c in range(len(sces))] for label in trY])
    tY = np.array([[1 if int_sce_map[c+1] == label else 0 for c in range(len(sces))] for label in tY])
        
    return (trX, trY, tX, tY)

In [6]:
import pickle

mfccs = pickle.load(open("mfccs.pickle", "rb"))

In [25]:
def get_ims(links):
    return np.array([(1 - cv2.imread(link, 0)/255).reshape((im_size, im_size, 1)) for link in links])
def get_wav(links):
    temp = []
    for link in links:
#         y, sr = librosa.load(link, sr = None, mono = True)
#         temp.append(librosa.feature.mfcc(y=y,sr=sr).T)
        temp.append(mfccs[link])
    return np.array(temp)
def get_wav2(links):
    temp = []
    for link in links:
#         y, sr = librosa.load(link, sr = None, mono = True)
#         temp.append(librosa.feature.mfcc(y=y,sr=sr).T)
        temp.append(mfccs2[link])
    return np.array(temp)

In [26]:
def data_loader(files, batch_size, file_type = 0):
    L = len(files)
    while True:
        batch_start = 0
        batch_end = batch_size
        
        while batch_start < L:
            lim = min(L, batch_end)
            if file_type == 0:
                X = get_ims([f[0] for f in files[batch_start:lim]])
                Y = np.array([f[1] for f in files[batch_start:lim]])
                yield (X, Y)
            elif file_type == 1:
                # wav files
                X = get_wav([f[0] for f in files[batch_start:lim]])
                Y = np.array([f[1] for f in files[batch_start:lim]])
                yield (X, Y)
            batch_start += batch_size
            batch_end += batch_size
            
def data_loader2(files, batch_size, file_type = 0):
    L = len(files)
    while True:
        batch_start = 0
        batch_end = batch_size
        
        while batch_start < L:
            lim = min(L, batch_end)
            if file_type == 0:
                X = get_ims(files[batch_start:lim])
                yield X
            elif file_type == 1:
                # wav files
                X = get_wav2(files[batch_start:lim])
                yield X
            batch_start += batch_size
            batch_end += batch_size

In [9]:
def make_model1D():
    model = Sequential()

    model.add(Conv1D(filters = 128, kernel_size = 100, input_shape=(313, 20), activation = 'relu'))
    model.add(Conv1D(filters = 128, kernel_size = 30, activation = 'relu'))
    model.add(MaxPooling1D(pool_size=2))
    
    model.add(Conv1D(filters = 128, kernel_size = 15, activation = 'relu'))
    model.add(MaxPooling1D(pool_size=2))
    
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_labels, activation = 'softmax'))

    adam = optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [10]:
def make_model2D_S():
    model = Sequential()

    model.add(Conv2D(filters = 32, kernel_size = (3, 3), input_shape=(im_size, im_size, 1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(rate = 0.5))
    model.add(Dense(units = n_labels, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
def make_model2D():
    model = Sequential()

    model.add(Conv2D(filters = 64, kernel_size = (3, 3), input_shape=(im_size, im_size, 1), activation = 'relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', dilation_rate = (3, 3)))
    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dropout(rate = 0.5))
    model.add(Dense(units = n_labels, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
def make_modelLSTM():
    model = Sequential()
    
    model.add(LSTM(units = 64, return_sequences = True, stateful = False, input_shape = (313, 20)))
    model.add(LSTM(units = 64, return_sequences = True, stateful = False))
    model.add(LSTM(units = 64, stateful = False))
    
    model.add(Dropout(rate = 0.5))
    model.add(Dense(units = n_labels, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
def make_modelCom():
    model = Sequential()

    model.add(Conv1D(filters = 32, kernel_size = 32, input_shape=(313, 20), activation = 'relu'))
    model.add(Conv1D(filters = 32, kernel_size = 32, activation = 'relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(LSTM(units = 64, return_sequences = True, stateful = False))
    model.add(LSTM(units = 64, return_sequences = True, stateful = False))
    model.add(LSTM(units = 64, stateful = False))

    model.add(Dropout(rate = 0.5))
    model.add(Dense(units = n_labels, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [14]:
def cnn(index, use_model, n_folds = 4, batch_size = 50, verbose = 1, sel_fold = 0, epochs = 100, save_model = False, val_split = 0.1, file_type = 0, name = None):
    random.seed(seed)
    model = use_model()
    
    trX, trY, tX, tY = get_sets(get_folds(index, n_folds), sel_fold)
    vsplit = int(len(tX) * val_split)
    val_X = tX[:vsplit]
    val_Y = tY[:vsplit]
    testX = tX[vsplit:]
    testY = tY[vsplit:]
                
    batch_per_epoch = int(math.ceil(len(trX) / batch_size))
    batch_per_val = int(math.ceil(len(val_X) / batch_size))
    batch_per_test = int(math.ceil(len(testX) / batch_size))
    
    if save_model:
        filepath = 'models/model_{index}.hdf5'.format(index = name)
        checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 0, save_best_only = True, mode = 'max')
        model.fit_generator(data_loader(list(zip(trX, trY)), batch_size, file_type),
                            steps_per_epoch = batch_per_epoch,
                            epochs = epochs, verbose = verbose,
                            callbacks = [checkpoint],
                            validation_data = data_loader(list(zip(val_X, val_Y)), batch_size, file_type),
                            validation_steps = batch_per_val)
    else:
        model.fit_generator(data_loader(list(zip(trX, trY)), batch_size, file_type),
                            steps_per_epoch = batch_per_epoch,
                            epochs = epochs,
                            verbose = verbose,
                            validation_data = data_loader(list(zip(val_X, val_Y)), batch_size, file_type),
                            validation_steps = batch_per_val)
    
    preds = model.predict_generator(data_loader(list(zip(testX, testY)), batch_size, file_type),
                                    steps = batch_per_test,
                                    verbose = verbose)
    
    predsT = np.argmax(preds, axis = 1)
    trueT = np.argmax(testY, axis = 1)
    print("Accuracy: {}. F1: {}".format(np.mean(np.equal(trueT, predsT)), f1_score(trueT, predsT, average = 'macro')))

In [15]:
from keras.models import load_model

def test_model(index, model, n_folds = 4, batch_size = 50, verbose = 1, sel_fold = 0, val_split = 0.1, file_type = 0):
    random.seed(seed)
    
    trX, trY, tX, tY = get_sets(get_folds(index, n_folds), sel_fold)
    vsplit = int(len(tX) * val_split)
    testX = tX[vsplit:]
    testY = tY[vsplit:]
                
    batch_per_test = int(math.ceil(len(testX) / batch_size))
        
    preds = model.predict_generator(data_loader(list(zip(testX, testY)), batch_size, file_type),
                                    steps = batch_per_test,
                                    verbose = verbose)
    
    predsT = np.argmax(preds, axis = 1)
    trueT = np.argmax(testY, axis = 1)
    print("Accuracy: {}. F1: {}".format(np.mean(np.equal(trueT, predsT)), f1_score(trueT, predsT, average = 'macro')))

In [36]:
def test_model_avg(indices, models, weights, file_types, n_folds = 4, batch_size = 50, verbose = 1, sel_fold = 0, val_split = 0.1):
    
    if not (len(indices) == len(models) == len(weights) == len(file_types)):
        print("Weights, models, file_types need to be the same length.")
        return
    if sum(weights) != 1:
        print("Sum of weights should equal 1")
        
    preds = None
    
    for index, model, weight, file_type in zip(indices, models, weights, file_types):
        
        if weight == 0:
            continue
    
        random.seed(seed)

        trX, trY, tX, tY = get_sets(get_folds(index, n_folds), sel_fold)
        vsplit = int(len(tX) * val_split)
        testX = tX[vsplit:]
        testY = tY[vsplit:]

        batch_per_test = int(math.ceil(len(testX) / batch_size))

        if preds is None:
            preds = weight * model.predict_generator(data_loader(list(zip(testX, testY)), batch_size, file_type),
                                                     steps = batch_per_test,
                                                     verbose = verbose)
        else:
            preds += weight * model.predict_generator(data_loader(list(zip(testX, testY)), batch_size, file_type),
                                                      steps = batch_per_test,
                                                      verbose = verbose)
    
    predsT = np.argmax(preds, axis = 1)
    trueT = np.argmax(testY, axis = 1)
    print("Accuracy: {}. F1: {}".format(np.mean(np.equal(trueT, predsT)), f1_score(trueT, predsT, average = 'macro')))
    print(confusion_matrix(trueT, predsT))
    print(f1_score(trueT, predsT, average = None))

In [17]:
def train_model(index, model, n_folds = 4, batch_size = 50, verbose = 1, sel_fold = 0, epochs = 100, save_model = False, val_split = 0.1, file_type = 0, name = None):
    random.seed(seed)
    
    trX, trY, tX, tY = get_sets(get_folds(index, n_folds), sel_fold)
    vsplit = int(len(tX) * val_split)
    val_X = tX[:vsplit]
    val_Y = tY[:vsplit]
    testX = tX[vsplit:]
    testY = tY[vsplit:]
    
    batch_per_epoch = int(math.ceil(len(trX) / batch_size))
    batch_per_val = int(math.ceil(len(val_X) / batch_size))
    batch_per_test = int(math.ceil(len(testX) / batch_size))
    
    if save_model:
        filepath = 'models/model_{index}.hdf5'.format(index = name)
        checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 0, save_best_only = True, mode = 'max')
        model.fit_generator(data_loader(list(zip(trX, trY)), batch_size, file_type),
                            steps_per_epoch = batch_per_epoch,
                            epochs = epochs, verbose = verbose,
                            callbacks = [checkpoint],
                            validation_data = data_loader(list(zip(val_X, val_Y)), batch_size, file_type),
                            validation_steps = batch_per_val)
    else:
        model.fit_generator(data_loader(list(zip(trX, trY)), batch_size, file_type),
                            steps_per_epoch = batch_per_epoch,
                            epochs = epochs,
                            verbose = verbose,
                            validation_data = data_loader(list(zip(val_X, val_Y)), batch_size, file_type),
                            validation_steps = batch_per_val)
    
    preds = model.predict_generator(data_loader(list(zip(testX, testY)), batch_size, file_type),
                                    steps = batch_per_test,
                                    verbose = verbose)
    
    predsT = np.argmax(preds, axis = 1)
    trueT = np.argmax(testY, axis = 1)
    print("Accuracy: {}. F1: {}".format(np.mean(np.equal(trueT, predsT)), f1_score(trueT, predsT, average = 'macro')))

In [31]:
mfccs2 = pickle.load(open("eval/eval/new/mfccs.pickle", "rb"))

In [69]:
def test_model_eval():
    weights = [0.3, 0.4, 0.3]
    
    ismap = {0:'absence', 1:'cooking', 2:'dishwashing', 3:'eating', 4:'other', 5:'social activity', 6:'vacuum cleaning', 7:'watching tv', 8:'working'}
    
    model1 = load_model('models/model_2D_small_3.hdf5')
    model2 = load_model('models/model_1D_norm.hdf5')
    model3 = load_model('models/model_LSTM_norm.hdf5')
        
    afs = [line.rstrip('\n') for line in open('eval/eval/meta.txt').readlines()]
    tlbatch = int(math.ceil(len(afs)/50))
    
    model1_eval_data = ['eval/eval/src/' + af[6:-4] + '_spec3.png' for af in afs]
    model23_eval_data = afs
    
    preds = weights[0] * model1.predict_generator(data_loader2(model1_eval_data, 50, 0),
                                                  steps = tlbatch,
                                                  verbose = 1)
    
    preds += weights[1] * model2.predict_generator(data_loader2(model23_eval_data, 50, 1),
                                                   steps = tlbatch,
                                                   verbose = 1)
    
    preds += weights[2] * model3.predict_generator(data_loader2(model23_eval_data, 50, 1),
                                                   steps = tlbatch,
                                                   verbose = 1)
    
    predsT = np.argmax(preds, axis = 1)
    return predsT
#     res = {af:ismap[pred] for af,pred in zip(afs,preds)}
#     return res

In [70]:
predsT = test_model_eval()

1460/1460 [==============================] - 180s 123ms/step


In [71]:
afs = [line.rstrip('\n') for line in open('eval/eval/meta.txt').readlines()]
ismap = {0:'absence', 1:'cooking', 2:'dishwashing', 3:'eating', 4:'other', 5:'social activity', 6:'vacuum cleaning', 7:'watching tv', 8:'working'}
res = {af:ismap[pred] for af,pred in zip(afs,predsT)}
with open('eval/eval/res2.txt', 'w') as f:
    for af in sorted(res.keys()):
        f.write(af + ' ' + res[af] + '\n')

In [17]:
cnn(0, make_model2D_S, epochs = 500, save_model = True, name = "2D_small_0")

Epoch 1/500
132/132 [==============================] - 143s 1s/step - loss: 1.6718 - acc: 0.3581 - val_loss: 1.4093 - val_acc: 0.4083
Epoch 2/500
132/132 [==============================] - 30s 229ms/step - loss: 1.4294 - acc: 0.4508 - val_loss: 1.2829 - val_acc: 0.4954
Epoch 3/500
132/132 [==============================] - 35s 264ms/step - loss: 1.3506 - acc: 0.4765 - val_loss: 1.2107 - val_acc: 0.5642
Epoch 4/500
132/132 [==============================] - 34s 261ms/step - loss: 1.2755 - acc: 0.5158 - val_loss: 1.2250 - val_acc: 0.5183
Epoch 5/500
132/132 [==============================] - 34s 257ms/step - loss: 1.2331 - acc: 0.5338 - val_loss: 1.1709 - val_acc: 0.5183
Epoch 6/500
132/132 [==============================] - 34s 257ms/step - loss: 1.2002 - acc: 0.5413 - val_loss: 1.1404 - val_acc: 0.5367
Epoch 7/500
132/132 [==============================] - 34s 254ms/step - loss: 1.1640 - acc: 0.5554 - val_loss: 1.1906 - val_acc: 0.5367
Epoch 8/500
132/132 [=============================

Epoch 61/500
132/132 [==============================] - 31s 237ms/step - loss: 0.3941 - acc: 0.8534 - val_loss: 2.1242 - val_acc: 0.5826
Epoch 62/500
132/132 [==============================] - 31s 234ms/step - loss: 0.3917 - acc: 0.8536 - val_loss: 2.2786 - val_acc: 0.5917
Epoch 63/500
132/132 [==============================] - 31s 234ms/step - loss: 0.3873 - acc: 0.8558 - val_loss: 2.3598 - val_acc: 0.5459
Epoch 64/500
132/132 [==============================] - 31s 234ms/step - loss: 0.3941 - acc: 0.8537 - val_loss: 2.4919 - val_acc: 0.5550
Epoch 65/500
132/132 [==============================] - 31s 234ms/step - loss: 0.3916 - acc: 0.8552 - val_loss: 2.3296 - val_acc: 0.6009
Epoch 66/500
132/132 [==============================] - 31s 235ms/step - loss: 0.3866 - acc: 0.8563 - val_loss: 2.2588 - val_acc: 0.5826
Epoch 67/500
132/132 [==============================] - 31s 235ms/step - loss: 0.3763 - acc: 0.8578 - val_loss: 2.2652 - val_acc: 0.5872
Epoch 68/500
132/132 [===================

132/132 [==============================] - 31s 234ms/step - loss: 0.2321 - acc: 0.9160 - val_loss: 3.1985 - val_acc: 0.6055
Epoch 180/500
132/132 [==============================] - 31s 234ms/step - loss: 0.2307 - acc: 0.9149 - val_loss: 3.3331 - val_acc: 0.5963
Epoch 181/500
132/132 [==============================] - 31s 233ms/step - loss: 0.2210 - acc: 0.9189 - val_loss: 3.6562 - val_acc: 0.5826
Epoch 182/500
132/132 [==============================] - 31s 236ms/step - loss: 0.2170 - acc: 0.9193 - val_loss: 3.6047 - val_acc: 0.5688
Epoch 183/500
132/132 [==============================] - 31s 235ms/step - loss: 0.2342 - acc: 0.9155 - val_loss: 3.3896 - val_acc: 0.5826
Epoch 184/500
132/132 [==============================] - 31s 237ms/step - loss: 0.2355 - acc: 0.9154 - val_loss: 3.1760 - val_acc: 0.5688
Epoch 185/500
132/132 [==============================] - 31s 233ms/step - loss: 0.2364 - acc: 0.9142 - val_loss: 3.4113 - val_acc: 0.5688
Epoch 186/500
132/132 [=========================

132/132 [==============================] - 31s 237ms/step - loss: 0.1807 - acc: 0.9383 - val_loss: 3.9368 - val_acc: 0.5734
Epoch 298/500
132/132 [==============================] - 31s 234ms/step - loss: 0.1840 - acc: 0.9370 - val_loss: 3.8765 - val_acc: 0.5688
Epoch 299/500
132/132 [==============================] - 31s 234ms/step - loss: 0.1805 - acc: 0.9370 - val_loss: 3.9428 - val_acc: 0.5826
Epoch 300/500
132/132 [==============================] - 31s 233ms/step - loss: 0.1834 - acc: 0.9348 - val_loss: 3.7680 - val_acc: 0.5963
Epoch 301/500
132/132 [==============================] - 31s 234ms/step - loss: 0.1802 - acc: 0.9358 - val_loss: 4.1115 - val_acc: 0.5734
Epoch 302/500
132/132 [==============================] - 31s 235ms/step - loss: 0.1667 - acc: 0.9381 - val_loss: 4.0665 - val_acc: 0.5596
Epoch 303/500
132/132 [==============================] - 31s 235ms/step - loss: 0.1621 - acc: 0.9425 - val_loss: 3.9224 - val_acc: 0.5780
Epoch 304/500
132/132 [=========================

132/132 [==============================] - 31s 234ms/step - loss: 0.1335 - acc: 0.9543 - val_loss: 4.0307 - val_acc: 0.5872
Epoch 416/500
132/132 [==============================] - 31s 233ms/step - loss: 0.1239 - acc: 0.9551 - val_loss: 4.0324 - val_acc: 0.5734
Epoch 417/500
132/132 [==============================] - 31s 233ms/step - loss: 0.1289 - acc: 0.9560 - val_loss: 4.0887 - val_acc: 0.5872
Epoch 418/500
132/132 [==============================] - 31s 233ms/step - loss: 0.1275 - acc: 0.9553 - val_loss: 4.1660 - val_acc: 0.6009
Epoch 419/500
132/132 [==============================] - 31s 232ms/step - loss: 0.1361 - acc: 0.9539 - val_loss: 3.7495 - val_acc: 0.5917
Epoch 420/500
132/132 [==============================] - 31s 233ms/step - loss: 0.1368 - acc: 0.9535 - val_loss: 3.8588 - val_acc: 0.5917
Epoch 421/500
132/132 [==============================] - 31s 232ms/step - loss: 0.1278 - acc: 0.9555 - val_loss: 4.3249 - val_acc: 0.5780
Epoch 422/500
132/132 [=========================

In [18]:
cnn(1, make_model2D_S, epochs = 500, save_model = True, name = "2D_small_1")

Epoch 1/500
132/132 [==============================] - 92s 695ms/step - loss: 2.1612 - acc: 0.1454 - val_loss: 1.9334 - val_acc: 0.2569
Epoch 2/500
132/132 [==============================] - 27s 204ms/step - loss: 1.5063 - acc: 0.4263 - val_loss: 1.2985 - val_acc: 0.4862
Epoch 3/500
132/132 [==============================] - 31s 232ms/step - loss: 1.1907 - acc: 0.5279 - val_loss: 1.1807 - val_acc: 0.5092
Epoch 4/500
132/132 [==============================] - 32s 239ms/step - loss: 1.0952 - acc: 0.5775 - val_loss: 1.0861 - val_acc: 0.5459
Epoch 5/500
132/132 [==============================] - 32s 240ms/step - loss: 1.0292 - acc: 0.6064 - val_loss: 1.2076 - val_acc: 0.5550
Epoch 6/500
132/132 [==============================] - 31s 238ms/step - loss: 0.9832 - acc: 0.6279 - val_loss: 0.9657 - val_acc: 0.6330
Epoch 7/500
132/132 [==============================] - 31s 237ms/step - loss: 0.9184 - acc: 0.6621 - val_loss: 0.9102 - val_acc: 0.6560
Epoch 8/500
132/132 [===========================

Epoch 61/500
132/132 [==============================] - 31s 235ms/step - loss: 0.1535 - acc: 0.9415 - val_loss: 0.8049 - val_acc: 0.7844
Epoch 62/500
132/132 [==============================] - 31s 235ms/step - loss: 0.1562 - acc: 0.9442 - val_loss: 0.8549 - val_acc: 0.7844
Epoch 63/500
132/132 [==============================] - 31s 237ms/step - loss: 0.1599 - acc: 0.9436 - val_loss: 0.8668 - val_acc: 0.7844
Epoch 64/500
132/132 [==============================] - 31s 235ms/step - loss: 0.1470 - acc: 0.9458 - val_loss: 0.9332 - val_acc: 0.7431
Epoch 65/500
132/132 [==============================] - 31s 235ms/step - loss: 0.1381 - acc: 0.9512 - val_loss: 0.7894 - val_acc: 0.8028
Epoch 66/500
132/132 [==============================] - 31s 236ms/step - loss: 0.1284 - acc: 0.9518 - val_loss: 0.8904 - val_acc: 0.7890
Epoch 67/500
132/132 [==============================] - 31s 235ms/step - loss: 0.1333 - acc: 0.9514 - val_loss: 1.0124 - val_acc: 0.7661
Epoch 68/500
132/132 [===================

132/132 [==============================] - 31s 234ms/step - loss: 0.0404 - acc: 0.9859 - val_loss: 1.7109 - val_acc: 0.7615
Epoch 180/500
132/132 [==============================] - 31s 234ms/step - loss: 0.0636 - acc: 0.9777 - val_loss: 1.5713 - val_acc: 0.7385
Epoch 181/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0403 - acc: 0.9844 - val_loss: 1.7166 - val_acc: 0.7569
Epoch 182/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0441 - acc: 0.9826 - val_loss: 1.5325 - val_acc: 0.7385
Epoch 183/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0401 - acc: 0.9864 - val_loss: 1.7144 - val_acc: 0.7248
Epoch 184/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0481 - acc: 0.9838 - val_loss: 1.6083 - val_acc: 0.7431
Epoch 185/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0449 - acc: 0.9839 - val_loss: 1.7882 - val_acc: 0.7477
Epoch 186/500
132/132 [=========================

132/132 [==============================] - 32s 239ms/step - loss: 0.0300 - acc: 0.9898 - val_loss: 1.9756 - val_acc: 0.7431
Epoch 298/500
132/132 [==============================] - 31s 238ms/step - loss: 0.0389 - acc: 0.9859 - val_loss: 2.0851 - val_acc: 0.7248
Epoch 299/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0222 - acc: 0.9920 - val_loss: 2.1150 - val_acc: 0.7385
Epoch 300/500
132/132 [==============================] - 31s 238ms/step - loss: 0.0331 - acc: 0.9894 - val_loss: 2.0452 - val_acc: 0.7248
Epoch 301/500
132/132 [==============================] - 31s 238ms/step - loss: 0.0242 - acc: 0.9908 - val_loss: 1.9959 - val_acc: 0.7569
Epoch 302/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0296 - acc: 0.9905 - val_loss: 2.1234 - val_acc: 0.7431
Epoch 303/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0424 - acc: 0.9861 - val_loss: 2.3179 - val_acc: 0.7523
Epoch 304/500
132/132 [=========================

132/132 [==============================] - 31s 237ms/step - loss: 0.0300 - acc: 0.9902 - val_loss: 1.9197 - val_acc: 0.7523
Epoch 416/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0296 - acc: 0.9898 - val_loss: 2.2117 - val_acc: 0.7569
Epoch 417/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0416 - acc: 0.9874 - val_loss: 2.1320 - val_acc: 0.7385
Epoch 418/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0298 - acc: 0.9901 - val_loss: 2.2242 - val_acc: 0.7431
Epoch 419/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0270 - acc: 0.9908 - val_loss: 2.2912 - val_acc: 0.7202
Epoch 420/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0538 - acc: 0.9827 - val_loss: 2.0445 - val_acc: 0.7706
Epoch 421/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0242 - acc: 0.9905 - val_loss: 2.1947 - val_acc: 0.7661
Epoch 422/500
132/132 [=========================

In [17]:
cnn(2, make_model2D_S, epochs = 500, save_model = True, name = "2D_small_2")

Epoch 1/500
132/132 [==============================] - 172s 1s/step - loss: 1.3186 - acc: 0.5022 - val_loss: 1.0449 - val_acc: 0.6055
Epoch 2/500
132/132 [==============================] - 28s 215ms/step - loss: 0.9194 - acc: 0.6523 - val_loss: 0.9006 - val_acc: 0.6743
Epoch 3/500
132/132 [==============================] - 29s 216ms/step - loss: 0.8205 - acc: 0.6957 - val_loss: 0.8033 - val_acc: 0.7110
Epoch 4/500
132/132 [==============================] - 28s 208ms/step - loss: 0.7478 - acc: 0.7182 - val_loss: 0.7884 - val_acc: 0.7202
Epoch 5/500
132/132 [==============================] - 28s 209ms/step - loss: 0.6895 - acc: 0.7419 - val_loss: 0.7391 - val_acc: 0.7523
Epoch 6/500
132/132 [==============================] - 27s 206ms/step - loss: 0.6544 - acc: 0.7503 - val_loss: 0.7000 - val_acc: 0.7569
Epoch 7/500
132/132 [==============================] - 28s 213ms/step - loss: 0.6083 - acc: 0.7774 - val_loss: 0.6733 - val_acc: 0.7523
Epoch 8/500
132/132 [=============================

Epoch 61/500
132/132 [==============================] - 31s 238ms/step - loss: 0.1061 - acc: 0.9615 - val_loss: 1.0096 - val_acc: 0.7982
Epoch 62/500
132/132 [==============================] - 32s 239ms/step - loss: 0.1043 - acc: 0.9620 - val_loss: 1.0908 - val_acc: 0.7844
Epoch 63/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0969 - acc: 0.9639 - val_loss: 1.0079 - val_acc: 0.8211
Epoch 64/500
132/132 [==============================] - 31s 236ms/step - loss: 0.1095 - acc: 0.9620 - val_loss: 0.9788 - val_acc: 0.8165
Epoch 65/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0935 - acc: 0.9670 - val_loss: 0.9999 - val_acc: 0.8165
Epoch 66/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0901 - acc: 0.9662 - val_loss: 1.1859 - val_acc: 0.8073
Epoch 67/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0876 - acc: 0.9688 - val_loss: 1.0028 - val_acc: 0.8257
Epoch 68/500
132/132 [===================

132/132 [==============================] - 31s 235ms/step - loss: 0.0579 - acc: 0.9824 - val_loss: 1.4679 - val_acc: 0.7890
Epoch 180/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0592 - acc: 0.9820 - val_loss: 1.5092 - val_acc: 0.7982
Epoch 181/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0468 - acc: 0.9864 - val_loss: 1.7540 - val_acc: 0.7798
Epoch 182/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0355 - acc: 0.9886 - val_loss: 1.5746 - val_acc: 0.7936
Epoch 183/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0362 - acc: 0.9874 - val_loss: 1.6865 - val_acc: 0.8028
Epoch 184/500
132/132 [==============================] - 32s 241ms/step - loss: 0.0311 - acc: 0.9885 - val_loss: 1.5245 - val_acc: 0.8073
Epoch 185/500
132/132 [==============================] - 32s 239ms/step - loss: 0.0399 - acc: 0.9879 - val_loss: 1.4458 - val_acc: 0.8303
Epoch 186/500
132/132 [=========================

132/132 [==============================] - 31s 236ms/step - loss: 0.0311 - acc: 0.9898 - val_loss: 1.7152 - val_acc: 0.8119
Epoch 298/500
132/132 [==============================] - 31s 235ms/step - loss: 0.0297 - acc: 0.9905 - val_loss: 1.6479 - val_acc: 0.8073
Epoch 299/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0353 - acc: 0.9903 - val_loss: 1.5942 - val_acc: 0.8303
Epoch 300/500
132/132 [==============================] - 31s 238ms/step - loss: 0.0289 - acc: 0.9908 - val_loss: 1.9137 - val_acc: 0.7936
Epoch 301/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0238 - acc: 0.9918 - val_loss: 1.9934 - val_acc: 0.7936
Epoch 302/500
132/132 [==============================] - 31s 236ms/step - loss: 0.0275 - acc: 0.9900 - val_loss: 1.9117 - val_acc: 0.7936
Epoch 303/500
132/132 [==============================] - 32s 240ms/step - loss: 0.0248 - acc: 0.9920 - val_loss: 1.8414 - val_acc: 0.7936
Epoch 304/500
132/132 [=========================

132/132 [==============================] - 32s 239ms/step - loss: 0.0229 - acc: 0.9927 - val_loss: 1.7211 - val_acc: 0.7890
Epoch 416/500
132/132 [==============================] - 31s 238ms/step - loss: 0.0287 - acc: 0.9900 - val_loss: 2.1544 - val_acc: 0.7798
Epoch 417/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0297 - acc: 0.9918 - val_loss: 1.9791 - val_acc: 0.7752
Epoch 418/500
132/132 [==============================] - 31s 237ms/step - loss: 0.0214 - acc: 0.9932 - val_loss: 2.1077 - val_acc: 0.7615
Epoch 419/500
132/132 [==============================] - 32s 240ms/step - loss: 0.0210 - acc: 0.9927 - val_loss: 1.7911 - val_acc: 0.8028
Epoch 420/500
132/132 [==============================] - 32s 241ms/step - loss: 0.0277 - acc: 0.9910 - val_loss: 1.8985 - val_acc: 0.8028
Epoch 421/500
132/132 [==============================] - 31s 238ms/step - loss: 0.0428 - acc: 0.9867 - val_loss: 2.0055 - val_acc: 0.7844
Epoch 422/500
132/132 [=========================

In [18]:
cnn(3, make_model2D_S, epochs = 500, save_model = True, name = "2D_small_3")

Epoch 1/500
132/132 [==============================] - 83s 628ms/step - loss: 1.6677 - acc: 0.3698 - val_loss: 1.1795 - val_acc: 0.5596
Epoch 2/500
132/132 [==============================] - 30s 228ms/step - loss: 1.0659 - acc: 0.5942 - val_loss: 0.9469 - val_acc: 0.6606
Epoch 3/500
132/132 [==============================] - 32s 242ms/step - loss: 0.9186 - acc: 0.6553 - val_loss: 0.8398 - val_acc: 0.6972
Epoch 4/500
132/132 [==============================] - 31s 237ms/step - loss: 0.8267 - acc: 0.6936 - val_loss: 0.7937 - val_acc: 0.7018
Epoch 5/500
132/132 [==============================] - 31s 237ms/step - loss: 0.7654 - acc: 0.7234 - val_loss: 0.7509 - val_acc: 0.7294
Epoch 6/500
132/132 [==============================] - 32s 244ms/step - loss: 0.7031 - acc: 0.7441 - val_loss: 0.6537 - val_acc: 0.7385
Epoch 7/500
132/132 [==============================] - 33s 248ms/step - loss: 0.6657 - acc: 0.7612 - val_loss: 0.5802 - val_acc: 0.7890
Epoch 8/500
132/132 [===========================

Epoch 61/500
132/132 [==============================] - 32s 240ms/step - loss: 0.1396 - acc: 0.9458 - val_loss: 0.5611 - val_acc: 0.8624
Epoch 62/500
132/132 [==============================] - 33s 251ms/step - loss: 0.1402 - acc: 0.9486 - val_loss: 0.4917 - val_acc: 0.8578
Epoch 63/500
132/132 [==============================] - 32s 243ms/step - loss: 0.1412 - acc: 0.9494 - val_loss: 0.6134 - val_acc: 0.8165
Epoch 64/500
132/132 [==============================] - 34s 261ms/step - loss: 0.1238 - acc: 0.9547 - val_loss: 0.5525 - val_acc: 0.8624
Epoch 65/500
132/132 [==============================] - 32s 245ms/step - loss: 0.1228 - acc: 0.9515 - val_loss: 0.5772 - val_acc: 0.8624
Epoch 66/500
132/132 [==============================] - 31s 236ms/step - loss: 0.1086 - acc: 0.9559 - val_loss: 0.6473 - val_acc: 0.8716
Epoch 67/500
132/132 [==============================] - 33s 250ms/step - loss: 0.1133 - acc: 0.9565 - val_loss: 0.5563 - val_acc: 0.8578
Epoch 68/500
132/132 [===================

132/132 [==============================] - 33s 247ms/step - loss: 0.0309 - acc: 0.9880 - val_loss: 1.0346 - val_acc: 0.8486
Epoch 180/500
132/132 [==============================] - 32s 246ms/step - loss: 0.0393 - acc: 0.9855 - val_loss: 0.8399 - val_acc: 0.8716
Epoch 181/500
132/132 [==============================] - 32s 245ms/step - loss: 0.0246 - acc: 0.9908 - val_loss: 1.0831 - val_acc: 0.8394
Epoch 182/500
132/132 [==============================] - 32s 246ms/step - loss: 0.0418 - acc: 0.9838 - val_loss: 1.0091 - val_acc: 0.8486
Epoch 183/500
132/132 [==============================] - 33s 247ms/step - loss: 0.0374 - acc: 0.9865 - val_loss: 0.8674 - val_acc: 0.8716
Epoch 184/500
132/132 [==============================] - 33s 247ms/step - loss: 0.0306 - acc: 0.9906 - val_loss: 1.0643 - val_acc: 0.8624
Epoch 185/500
132/132 [==============================] - 32s 245ms/step - loss: 0.0376 - acc: 0.9855 - val_loss: 0.8712 - val_acc: 0.8853
Epoch 186/500
132/132 [=========================

132/132 [==============================] - 32s 239ms/step - loss: 0.0353 - acc: 0.9897 - val_loss: 0.8590 - val_acc: 0.8853
Epoch 298/500
132/132 [==============================] - 31s 239ms/step - loss: 0.0350 - acc: 0.9908 - val_loss: 0.9297 - val_acc: 0.8624
Epoch 299/500
132/132 [==============================] - 32s 241ms/step - loss: 0.0263 - acc: 0.9912 - val_loss: 0.8815 - val_acc: 0.8945
Epoch 300/500
132/132 [==============================] - 32s 241ms/step - loss: 0.0283 - acc: 0.9911 - val_loss: 0.9500 - val_acc: 0.8761
Epoch 301/500
132/132 [==============================] - 32s 241ms/step - loss: 0.0300 - acc: 0.9900 - val_loss: 0.9754 - val_acc: 0.8899
Epoch 302/500
132/132 [==============================] - 32s 242ms/step - loss: 0.0319 - acc: 0.9875 - val_loss: 1.2514 - val_acc: 0.8578
Epoch 303/500
132/132 [==============================] - 32s 239ms/step - loss: 0.0218 - acc: 0.9912 - val_loss: 1.0164 - val_acc: 0.8899
Epoch 304/500
132/132 [=========================

132/132 [==============================] - 33s 249ms/step - loss: 0.0160 - acc: 0.9939 - val_loss: 0.9284 - val_acc: 0.8991
Epoch 416/500
132/132 [==============================] - 32s 239ms/step - loss: 0.0255 - acc: 0.9923 - val_loss: 1.0982 - val_acc: 0.8624
Epoch 417/500
132/132 [==============================] - 32s 239ms/step - loss: 0.0400 - acc: 0.9888 - val_loss: 0.8111 - val_acc: 0.8991
Epoch 418/500
132/132 [==============================] - 32s 240ms/step - loss: 0.0164 - acc: 0.9944 - val_loss: 0.9451 - val_acc: 0.8991
Epoch 419/500
132/132 [==============================] - 32s 242ms/step - loss: 0.0290 - acc: 0.9898 - val_loss: 0.8581 - val_acc: 0.8899
Epoch 420/500
132/132 [==============================] - 32s 245ms/step - loss: 0.0163 - acc: 0.9941 - val_loss: 0.8713 - val_acc: 0.8761
Epoch 421/500
132/132 [==============================] - 32s 241ms/step - loss: 0.0196 - acc: 0.9942 - val_loss: 0.8478 - val_acc: 0.8991
Epoch 422/500
132/132 [=========================

In [29]:
cnn(4, make_model1D, epochs = 500, save_model = True, name = "1D_norm", file_type = 1)

Epoch 1/500
132/132 [==============================] - 78s 588ms/step - loss: 6.0036 - acc: 0.2390 - val_loss: 1.4845 - val_acc: 0.4174
Epoch 2/500
132/132 [==============================] - 81s 615ms/step - loss: 1.6086 - acc: 0.3918 - val_loss: 1.3040 - val_acc: 0.4358
Epoch 3/500
132/132 [==============================] - 81s 613ms/step - loss: 1.4592 - acc: 0.4322 - val_loss: 1.2189 - val_acc: 0.4862
Epoch 4/500
132/132 [==============================] - 77s 586ms/step - loss: 1.3583 - acc: 0.4678 - val_loss: 1.1723 - val_acc: 0.5505
Epoch 5/500
132/132 [==============================] - 77s 587ms/step - loss: 1.3060 - acc: 0.5016 - val_loss: 1.1102 - val_acc: 0.5321
Epoch 6/500
132/132 [==============================] - 78s 593ms/step - loss: 1.2664 - acc: 0.5153 - val_loss: 1.0963 - val_acc: 0.5550
Epoch 7/500
132/132 [==============================] - 77s 585ms/step - loss: 1.2230 - acc: 0.5340 - val_loss: 1.0858 - val_acc: 0.5642
Epoch 8/500
132/132 [===========================

Epoch 61/500
132/132 [==============================] - 79s 598ms/step - loss: 0.4682 - acc: 0.8164 - val_loss: 0.5651 - val_acc: 0.7936
Epoch 62/500
132/132 [==============================] - 84s 638ms/step - loss: 0.4574 - acc: 0.8252 - val_loss: 0.5637 - val_acc: 0.7890
Epoch 63/500
132/132 [==============================] - 83s 631ms/step - loss: 0.4519 - acc: 0.8218 - val_loss: 0.5568 - val_acc: 0.8119
Epoch 64/500
132/132 [==============================] - 85s 642ms/step - loss: 0.4478 - acc: 0.8270 - val_loss: 0.5784 - val_acc: 0.7752
Epoch 65/500
132/132 [==============================] - 83s 629ms/step - loss: 0.4409 - acc: 0.8288 - val_loss: 0.5776 - val_acc: 0.8028
Epoch 66/500
132/132 [==============================] - 83s 633ms/step - loss: 0.4220 - acc: 0.8357 - val_loss: 0.5574 - val_acc: 0.8119
Epoch 67/500
132/132 [==============================] - 81s 612ms/step - loss: 0.4231 - acc: 0.8355 - val_loss: 0.5582 - val_acc: 0.7890
Epoch 68/500
132/132 [===================

132/132 [==============================] - 79s 599ms/step - loss: 0.1231 - acc: 0.9571 - val_loss: 0.5077 - val_acc: 0.8853
Epoch 180/500
132/132 [==============================] - 79s 600ms/step - loss: 0.1222 - acc: 0.9588 - val_loss: 0.5724 - val_acc: 0.8578
Epoch 181/500
132/132 [==============================] - 79s 598ms/step - loss: 0.1192 - acc: 0.9585 - val_loss: 0.5471 - val_acc: 0.8440
Epoch 182/500
132/132 [==============================] - 78s 592ms/step - loss: 0.1132 - acc: 0.9616 - val_loss: 0.5525 - val_acc: 0.8257
Epoch 183/500
132/132 [==============================] - 78s 592ms/step - loss: 0.1197 - acc: 0.9570 - val_loss: 0.5516 - val_acc: 0.8532
Epoch 184/500
132/132 [==============================] - 78s 590ms/step - loss: 0.1085 - acc: 0.9618 - val_loss: 0.5758 - val_acc: 0.8532
Epoch 185/500
132/132 [==============================] - 78s 590ms/step - loss: 0.1101 - acc: 0.9617 - val_loss: 0.5719 - val_acc: 0.8716
Epoch 186/500
132/132 [=========================

132/132 [==============================] - 78s 593ms/step - loss: 0.0347 - acc: 0.9906 - val_loss: 0.5426 - val_acc: 0.8899
Epoch 298/500
132/132 [==============================] - 79s 597ms/step - loss: 0.0312 - acc: 0.9921 - val_loss: 0.5573 - val_acc: 0.8532
Epoch 299/500
132/132 [==============================] - 78s 594ms/step - loss: 0.0351 - acc: 0.9886 - val_loss: 0.4975 - val_acc: 0.8853
Epoch 300/500
132/132 [==============================] - 79s 596ms/step - loss: 0.0302 - acc: 0.9923 - val_loss: 0.6258 - val_acc: 0.9128
Epoch 301/500
132/132 [==============================] - 78s 590ms/step - loss: 0.0350 - acc: 0.9900 - val_loss: 0.5320 - val_acc: 0.8716
Epoch 302/500
132/132 [==============================] - 78s 595ms/step - loss: 0.0346 - acc: 0.9892 - val_loss: 0.5615 - val_acc: 0.8991
Epoch 303/500
132/132 [==============================] - 79s 597ms/step - loss: 0.0316 - acc: 0.9897 - val_loss: 0.6170 - val_acc: 0.8578
Epoch 304/500
132/132 [=========================

132/132 [==============================] - 78s 594ms/step - loss: 0.0165 - acc: 0.9959 - val_loss: 0.6114 - val_acc: 0.8761
Epoch 416/500
132/132 [==============================] - 78s 593ms/step - loss: 0.0116 - acc: 0.9970 - val_loss: 0.6662 - val_acc: 0.8624
Epoch 417/500
132/132 [==============================] - 78s 590ms/step - loss: 0.0173 - acc: 0.9947 - val_loss: 0.5744 - val_acc: 0.8807
Epoch 418/500
132/132 [==============================] - 79s 596ms/step - loss: 0.0149 - acc: 0.9952 - val_loss: 0.7394 - val_acc: 0.8853
Epoch 419/500
132/132 [==============================] - 78s 594ms/step - loss: 0.0186 - acc: 0.9945 - val_loss: 0.6595 - val_acc: 0.8807
Epoch 420/500
132/132 [==============================] - 80s 604ms/step - loss: 0.0142 - acc: 0.9959 - val_loss: 0.6941 - val_acc: 0.9037
Epoch 421/500
132/132 [==============================] - 78s 591ms/step - loss: 0.0187 - acc: 0.9944 - val_loss: 0.7151 - val_acc: 0.8532
Epoch 422/500
132/132 [=========================

KeyboardInterrupt: 

In [26]:
model = load_model("models/model_1D_norm.hdf5")
train_model(4, model, file_type = 1, epochs = 100, save_model = True, name = "1D_norm")

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Epoch 1/100
132/132 [==============================] - 78s 593ms/step - loss: 0.0358 - acc: 0.9879 - val_loss: 0.5196 - val_acc: 0.8991
Epoch 2/100
132/132 [==============================] - 78s 593ms/step - loss: 0.0368 - acc: 0.9889 - val_loss: 0.5279 - val_acc: 0.8670
Epoch 3/100
132/132 [==============================] - 77s 583ms/step - loss: 0.0329 - acc: 0.9905 - val_loss: 0.5390 - val_acc: 0.8991
Epoch 4/100
132/132 [==============================] - 77s 584ms/step - loss: 0.0324 - acc: 0.9889 - val_loss: 0.6341 - val_acc: 0.8670
Epoch 5/100
132/132 [==============================] - 77s 584ms/step - loss: 0.0332 - acc: 0.9908 - val_loss: 0.6258 - val_acc: 0.8807
Epoch 6/100
132/132 [==============================] - 78s 593ms/step - loss: 0.0402 - acc: 0.9882 - val_loss: 0.5604 - val_acc: 0.8670
Epoch 7/100
132/132 [==============================] - 79s 598ms/step - loss: 0.0250 - acc: 0.9948 - 

In [27]:
cnn(4, make_modelLSTM, epochs = 500, save_model = True, name = "LSTM_norm", file_type = 1)

Epoch 1/500
132/132 [==============================] - 70s 529ms/step - loss: 1.4877 - acc: 0.4409 - val_loss: 1.0417 - val_acc: 0.5963
Epoch 2/500
132/132 [==============================] - 68s 516ms/step - loss: 0.9838 - acc: 0.6325 - val_loss: 0.8893 - val_acc: 0.6239
Epoch 3/500
132/132 [==============================] - 68s 515ms/step - loss: 0.9222 - acc: 0.6501 - val_loss: 0.9605 - val_acc: 0.6376
Epoch 4/500
132/132 [==============================] - 68s 514ms/step - loss: 0.8451 - acc: 0.6859 - val_loss: 0.8228 - val_acc: 0.7018
Epoch 5/500
132/132 [==============================] - 68s 516ms/step - loss: 0.7673 - acc: 0.7232 - val_loss: 0.8217 - val_acc: 0.6835
Epoch 6/500
132/132 [==============================] - 68s 516ms/step - loss: 0.8932 - acc: 0.6740 - val_loss: 0.8068 - val_acc: 0.6835
Epoch 7/500
132/132 [==============================] - 68s 515ms/step - loss: 0.7783 - acc: 0.7223 - val_loss: 0.7522 - val_acc: 0.7156
Epoch 8/500
132/132 [===========================

Epoch 61/500
132/132 [==============================] - 68s 517ms/step - loss: 0.4760 - acc: 0.8359 - val_loss: 0.6226 - val_acc: 0.7752
Epoch 62/500
132/132 [==============================] - 69s 522ms/step - loss: 0.4610 - acc: 0.8434 - val_loss: 0.5040 - val_acc: 0.8211
Epoch 63/500
132/132 [==============================] - 70s 530ms/step - loss: 0.4029 - acc: 0.8592 - val_loss: 0.5011 - val_acc: 0.8028
Epoch 64/500
132/132 [==============================] - 68s 519ms/step - loss: 0.4232 - acc: 0.8565 - val_loss: 0.5381 - val_acc: 0.8165
Epoch 65/500
132/132 [==============================] - 69s 520ms/step - loss: 0.4261 - acc: 0.8551 - val_loss: 0.5612 - val_acc: 0.8165
Epoch 66/500
132/132 [==============================] - 68s 518ms/step - loss: 0.4422 - acc: 0.8494 - val_loss: 0.5171 - val_acc: 0.8165
Epoch 67/500
132/132 [==============================] - 69s 521ms/step - loss: 0.4290 - acc: 0.8598 - val_loss: 0.5535 - val_acc: 0.8073
Epoch 68/500
132/132 [===================

132/132 [==============================] - 68s 519ms/step - loss: 0.2634 - acc: 0.9094 - val_loss: 0.5041 - val_acc: 0.8394
Epoch 180/500
132/132 [==============================] - 68s 519ms/step - loss: 0.2759 - acc: 0.9041 - val_loss: 0.3754 - val_acc: 0.8853
Epoch 181/500
132/132 [==============================] - 69s 522ms/step - loss: 0.2632 - acc: 0.9073 - val_loss: 0.4349 - val_acc: 0.8624
Epoch 182/500
132/132 [==============================] - 68s 518ms/step - loss: 0.2417 - acc: 0.9136 - val_loss: 0.5491 - val_acc: 0.8578
Epoch 183/500
132/132 [==============================] - 68s 518ms/step - loss: 0.2602 - acc: 0.9082 - val_loss: 0.4700 - val_acc: 0.8761
Epoch 184/500
132/132 [==============================] - 69s 519ms/step - loss: 0.2167 - acc: 0.9227 - val_loss: 0.3934 - val_acc: 0.8945
Epoch 185/500
132/132 [==============================] - 67s 511ms/step - loss: 0.2021 - acc: 0.9242 - val_loss: 0.3818 - val_acc: 0.8853
Epoch 186/500
132/132 [=========================

132/132 [==============================] - 67s 507ms/step - loss: 0.1585 - acc: 0.9455 - val_loss: 0.6150 - val_acc: 0.8578
Epoch 298/500
132/132 [==============================] - 67s 509ms/step - loss: 0.1285 - acc: 0.9545 - val_loss: 0.4617 - val_acc: 0.8807
Epoch 299/500
132/132 [==============================] - 67s 509ms/step - loss: 0.1356 - acc: 0.9524 - val_loss: 0.6146 - val_acc: 0.8486
Epoch 300/500
132/132 [==============================] - 67s 509ms/step - loss: 0.0977 - acc: 0.9648 - val_loss: 0.6657 - val_acc: 0.8624
Epoch 301/500
132/132 [==============================] - 67s 508ms/step - loss: 0.1231 - acc: 0.9573 - val_loss: 0.6698 - val_acc: 0.8394
Epoch 302/500
132/132 [==============================] - 67s 508ms/step - loss: 0.1180 - acc: 0.9570 - val_loss: 0.6192 - val_acc: 0.8440
Epoch 303/500
132/132 [==============================] - 68s 513ms/step - loss: 0.1116 - acc: 0.9603 - val_loss: 0.6135 - val_acc: 0.8578
Epoch 304/500
132/132 [=========================

132/132 [==============================] - 67s 506ms/step - loss: 0.0389 - acc: 0.9882 - val_loss: 0.6367 - val_acc: 0.8991
Epoch 416/500
132/132 [==============================] - 67s 506ms/step - loss: 0.0274 - acc: 0.9917 - val_loss: 0.6392 - val_acc: 0.8761
Epoch 417/500
132/132 [==============================] - 67s 506ms/step - loss: 0.0566 - acc: 0.9800 - val_loss: 0.5896 - val_acc: 0.8945
Epoch 418/500
132/132 [==============================] - 66s 504ms/step - loss: 0.0305 - acc: 0.9906 - val_loss: 0.6935 - val_acc: 0.8991
Epoch 419/500
132/132 [==============================] - 67s 505ms/step - loss: 0.1253 - acc: 0.9670 - val_loss: 0.6264 - val_acc: 0.8670
Epoch 420/500
132/132 [==============================] - 67s 508ms/step - loss: 0.0621 - acc: 0.9786 - val_loss: 0.5776 - val_acc: 0.8578
Epoch 421/500
132/132 [==============================] - 67s 506ms/step - loss: 0.0985 - acc: 0.9703 - val_loss: 0.4829 - val_acc: 0.8761
Epoch 422/500
132/132 [=========================

In [28]:
cnn(4, make_modelCom, epochs = 500, save_model = True, name = "Com_norm", file_type = 1)

Epoch 1/500
132/132 [==============================] - 40s 306ms/step - loss: 1.6890 - acc: 0.3368 - val_loss: 1.4291 - val_acc: 0.4037
Epoch 2/500
132/132 [==============================] - 38s 289ms/step - loss: 1.3679 - acc: 0.4458 - val_loss: 1.3360 - val_acc: 0.4037
Epoch 3/500
132/132 [==============================] - 45s 342ms/step - loss: 1.4274 - acc: 0.4179 - val_loss: 1.1894 - val_acc: 0.4725
Epoch 4/500
132/132 [==============================] - 45s 340ms/step - loss: 1.3364 - acc: 0.4626 - val_loss: 1.2203 - val_acc: 0.4541
Epoch 5/500
132/132 [==============================] - 45s 338ms/step - loss: 1.3266 - acc: 0.4432 - val_loss: 1.1925 - val_acc: 0.4587
Epoch 6/500
132/132 [==============================] - 45s 340ms/step - loss: 1.2993 - acc: 0.4669 - val_loss: 1.2508 - val_acc: 0.4495
Epoch 7/500
132/132 [==============================] - 45s 341ms/step - loss: 1.2630 - acc: 0.4908 - val_loss: 1.4301 - val_acc: 0.3853
Epoch 8/500
132/132 [===========================

Epoch 61/500
132/132 [==============================] - 56s 427ms/step - loss: 0.8375 - acc: 0.6758 - val_loss: 0.9195 - val_acc: 0.5963
Epoch 62/500
132/132 [==============================] - 58s 441ms/step - loss: 0.7694 - acc: 0.7097 - val_loss: 0.9009 - val_acc: 0.6193
Epoch 63/500
132/132 [==============================] - 60s 451ms/step - loss: 0.8416 - acc: 0.6769 - val_loss: 0.9258 - val_acc: 0.5963
Epoch 64/500
132/132 [==============================] - 58s 440ms/step - loss: 0.8206 - acc: 0.6791 - val_loss: 0.8955 - val_acc: 0.6193
Epoch 65/500
132/132 [==============================] - 57s 430ms/step - loss: 0.7847 - acc: 0.6961 - val_loss: 0.9065 - val_acc: 0.6193
Epoch 66/500
132/132 [==============================] - 58s 436ms/step - loss: 0.8397 - acc: 0.6748 - val_loss: 1.1287 - val_acc: 0.5596
Epoch 67/500
132/132 [==============================] - 58s 439ms/step - loss: 0.8644 - acc: 0.6656 - val_loss: 0.9116 - val_acc: 0.6101
Epoch 68/500
132/132 [===================

132/132 [==============================] - 46s 348ms/step - loss: 0.5281 - acc: 0.8151 - val_loss: 0.6524 - val_acc: 0.7477
Epoch 180/500
132/132 [==============================] - 46s 348ms/step - loss: 0.6074 - acc: 0.7892 - val_loss: 0.6840 - val_acc: 0.7202
Epoch 181/500
132/132 [==============================] - 46s 349ms/step - loss: 0.5317 - acc: 0.8089 - val_loss: 0.6546 - val_acc: 0.7339
Epoch 182/500
132/132 [==============================] - 46s 347ms/step - loss: 0.5505 - acc: 0.8050 - val_loss: 0.6917 - val_acc: 0.7431
Epoch 183/500
132/132 [==============================] - 46s 352ms/step - loss: 0.5924 - acc: 0.7869 - val_loss: 0.6994 - val_acc: 0.7477
Epoch 184/500
132/132 [==============================] - 46s 348ms/step - loss: 0.5305 - acc: 0.8139 - val_loss: 0.6785 - val_acc: 0.7477
Epoch 185/500
132/132 [==============================] - 46s 350ms/step - loss: 0.5468 - acc: 0.8087 - val_loss: 0.6820 - val_acc: 0.7431
Epoch 186/500
132/132 [=========================

132/132 [==============================] - 45s 340ms/step - loss: 0.4754 - acc: 0.8356 - val_loss: 0.6961 - val_acc: 0.7477
Epoch 298/500
132/132 [==============================] - 45s 338ms/step - loss: 0.4563 - acc: 0.8309 - val_loss: 0.6992 - val_acc: 0.7477
Epoch 299/500
132/132 [==============================] - 45s 338ms/step - loss: 0.4541 - acc: 0.8365 - val_loss: 0.6895 - val_acc: 0.7523
Epoch 300/500
132/132 [==============================] - 45s 338ms/step - loss: 0.4554 - acc: 0.8392 - val_loss: 0.6819 - val_acc: 0.7385
Epoch 301/500
132/132 [==============================] - 45s 339ms/step - loss: 0.4813 - acc: 0.8305 - val_loss: 0.6697 - val_acc: 0.7385
Epoch 302/500
132/132 [==============================] - 45s 338ms/step - loss: 0.6450 - acc: 0.7768 - val_loss: 0.7840 - val_acc: 0.7248
Epoch 303/500
132/132 [==============================] - 45s 338ms/step - loss: 0.5898 - acc: 0.7907 - val_loss: 0.7193 - val_acc: 0.7248
Epoch 304/500
132/132 [=========================

132/132 [==============================] - 43s 329ms/step - loss: 0.5380 - acc: 0.8039 - val_loss: 0.6668 - val_acc: 0.7752
Epoch 416/500
132/132 [==============================] - 43s 329ms/step - loss: 0.5497 - acc: 0.8008 - val_loss: 0.6681 - val_acc: 0.7523
Epoch 417/500
132/132 [==============================] - 43s 327ms/step - loss: 0.5513 - acc: 0.7935 - val_loss: 0.6763 - val_acc: 0.7523
Epoch 418/500
132/132 [==============================] - 43s 329ms/step - loss: 0.6049 - acc: 0.7739 - val_loss: 0.7467 - val_acc: 0.7339
Epoch 419/500
132/132 [==============================] - 43s 329ms/step - loss: 0.5940 - acc: 0.7824 - val_loss: 0.8109 - val_acc: 0.7018
Epoch 420/500
132/132 [==============================] - 44s 335ms/step - loss: 0.6068 - acc: 0.7731 - val_loss: 0.6997 - val_acc: 0.7385
Epoch 421/500
132/132 [==============================] - 44s 334ms/step - loss: 0.6169 - acc: 0.7724 - val_loss: 0.7508 - val_acc: 0.7202
Epoch 422/500
132/132 [=========================

In [38]:
models = [load_model('models/model_2D_small_0.hdf5'), 
          load_model('models/model_2D_small_1.hdf5'), 
          load_model('models/model_2D_small_2.hdf5'), 
          load_model('models/model_2D_small_3.hdf5'), 
          load_model('models/model_1D_norm.hdf5'), 
          load_model('models/model_LSTM_norm.hdf5'), 
          load_model('models/model_Com_norm.hdf5')]

In [19]:
test_model(0, models[0])
test_model(1, models[1])
test_model(2, models[2])
test_model(3, models[3])
test_model(4, models[4], file_type = 1)
test_model(4, models[5], file_type = 1)
test_model(4, models[6], file_type = 1)

40/40 [==============================] - 3s 71ms/step
Accuracy: 0.5642458100558659. F1: 0.5559214378924485
40/40 [==============================] - 3s 70ms/step
Accuracy: 0.7953275774504824. F1: 0.7944290220418879
40/40 [==============================] - 21s 517ms/step
Accuracy: 0.8110716099542915. F1: 0.8099749109721136
40/40 [==============================] - 22s 548ms/step
Accuracy: 0.8689690198070086. F1: 0.865455499361151
40/40 [==============================] - 7s 167ms/step
Accuracy: 0.8969019807008634. F1: 0.8958705030987514
40/40 [==============================] - 5s 128ms/step
Accuracy: 0.8806500761808025. F1: 0.8800810128115094
40/40 [==============================] - 4s 108ms/step
Accuracy: 0.8364652107668867. F1: 0.8318890576416703


In [39]:
indices = [0, 1, 2, 3, 4, 4, 4]
avg_models = models
weights = [0, 0, 0, 0, .5, .5, 0]
file_types = [0, 0, 0, 0, 1, 1, 1]
test_model_avg(indices, avg_models, weights, file_types)

40/40 [==============================] - 5s 115ms/step
Accuracy: 0.9116302691721686. F1: 0.9107363152807513
[[192   0   0   0  10   1   0   0  18]
 [  0 214   7   0   5   0   0   0   0]
 [  0   0 211   1   4   2   0   0   2]
 [  2   0   4 197   7   1   0   1   4]
 [ 29   3   2   2 159   5   0   0  12]
 [  0   0   2   0   4 220   0   0   0]
 [  0   0   0   0   0   0 222   0   0]
 [  0   0   0   1   0   0   0 213   0]
 [ 20   0   1   3  21   0   0   0 167]]
[0.82758621 0.96613995 0.94407159 0.93809524 0.7535545  0.96703297
 1.         0.9953271  0.80481928]


In [33]:
indices = [0, 1, 2, 3, 4, 4, 4]
avg_models = models
weights = [0, 0, 0, .3, .4, .3, 0]
file_types = [0, 0, 0, 0, 1, 1, 1]
test_model_avg(indices, avg_models, weights, file_types)

40/40 [==============================] - 4s 98ms/step
Accuracy: 0.9233113255459624. F1: 0.9219258532903954


In [23]:
indices = [0, 1, 2, 3, 4, 4, 4]
avg_models = models
weights = [0, 0, 0, .3, .4, .3, 0]
file_types = [0, 0, 0, 0, 1, 1, 1]
cm = test_model_avg(indices, avg_models, weights, file_types)

40/40 [==============================] - 5s 125ms/step
Accuracy: 0.9233113255459624. F1: 0.9219258532903954


In [24]:
print(cm)

[0.84453782 0.97091723 0.95982143 0.93023256 0.7761194  0.97995546
 1.         1.         0.83574879]


In [28]:
print(cm)

[[201   0   0   0   7   1   0   0  12]
 [  0 217   7   0   1   0   0   0   1]
 [  0   0 215   3   0   1   0   0   1]
 [  2   0   2 200   8   0   0   0   4]
 [ 35   3   0   6 156   1   0   0  11]
 [  0   0   2   1   3 220   0   0   0]
 [  0   0   0   0   0   0 222   0   0]
 [  0   0   0   0   0   0   0 214   0]
 [ 17   1   2   4  15   0   0   0 173]]
